<h1>Data Checking & Sorting</h1>

In [0]:
# Pandas for reading .csv
import pandas as pd
import numpy as np
from datetime import timedelta

In [0]:
df = pd.read_csv('order_brush_order.csv')
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [0]:
# No empty data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [0]:
df.isnull().sum()

orderid       0
shopid        0
userid        0
event_time    0
dtype: int64

In [0]:
frequents = df['userid'].value_counts()
frequents[:20]

124597967    97
50167978     63
78206381     49
145795810    47
95399773     46
198954492    46
85650202     44
153541865    41
95500119     41
153538082    41
153541705    41
153541920    40
26107753     40
134730965    40
153541782    40
211166728    39
98620510     39
95399915     38
95703687     38
188025647    37
Name: userid, dtype: int64

In [0]:
df_sorted = df
df_sorted['event_time'] = pd.to_datetime(df_sorted['event_time'])
df_sorted = df_sorted.sort_values(by=['shopid','event_time'], ignore_index=True)
df_sorted.head(20)

,orderid,shopid,userid,event_time
0,31086409141107,10009,196962305,2019-12-27 03:06:50
1,31144571933461,10051,2854032,2019-12-27 19:16:11
2,31254979546679,10051,48600461,2019-12-29 01:56:19
3,31195675919209,10061,168750452,2019-12-28 09:27:55
4,31205132327893,10061,194819216,2019-12-28 12:05:32
5,31412115824794,10061,130633421,2019-12-30 21:35:15
6,31431527100615,10061,62464559,2019-12-31 02:58:48
7,31075686185309,10084,4401933,2019-12-27 00:08:06
8,31077155357404,10084,13837190,2019-12-27 00:32:36
9,31079024994425,10084,39828049,2019-12-27 01:03:44


In [0]:
# If shop event < 3 then userid = 0
#
#
#

<h1>Function Preparations</h1>

In [0]:
shops = df_sorted['shopid'].unique()
len(shops)

18770

In [0]:
df_sorted['event_time'][2] - df_sorted['event_time'][1] < timedelta(2)

True

<h1>Check for Suspicious Shops</h1>

In [0]:
def hourdelta(time1, time2):
    '''Calculate the difference between
    time1 and time2 (time1-time2) in hours.'''
    delta = (time1 - time2).total_seconds()
    if(delta<0):
        delta = delta*(-1)
    return delta/3600

def check_brush(df):
  store = []
  for i in shops:
    check = df[df['shopid'] == i]
    check = check['event_time']
    check = check.reset_index(drop=True)
    if (len(check) < 3):
      store.append(0)
    else:
      cek = False
      for j in range(0,len(check)-2):
        if (hourdelta(check[j+2], check[j]) < 1):
          cek = True
      if cek == False:
        store.append(0)
      else:
        store.append(1)
  return store

store = check_brush(df_sorted)

In [0]:
def user_check(df, shops, store):
  user = []
  for i in range(len(store)):
    string = ''
    if store[i] == 0:
      user.append(0)
    else:
      a = 0
      check = df[df['shopid'] == shops[i]]
      users = check['userid'].value_counts().keys().tolist()
      count = check['userid'].value_counts().tolist()
      the_max = max(count)
      for j in range(len(count)):
        if count[j] == the_max:
          if a == 0:
            a = 1
            string += str(users[j])
          else:
            string += "&"
            string += str(users[j])
      user.append(string)
  return user

frauds = user_check(df_sorted, shops, store)

In [0]:
# Result DataFrame

result = pd.DataFrame(columns=['shopid','userid'])

In [0]:
# Links
# https://stackoverflow.com/questions/2119472/convert-a-timedelta-to-days-hours-and-minutes

In [0]:
# Result DataFrame

result = pd.DataFrame(columns=['shopid','userid'])
result['shopid'] = df_sorted['shopid'].unique()
result['userid'] = frauds

# New Section

In [0]:
result.to_csv('Submission3.csv', index=False)